In [ ]:
#init libs
import math
import csv
import numpy as np
from numpy import loadtxt
from tensorflow.python.keras import Input, Model, Sequential
from tensorflow.python.keras.layers import  Concatenate, Activation, Dropout, Flatten, Dense, LSTM,Conv1D, MaxPooling1D, Reshape, Permute,BatchNormalization
from tensorflow.python.keras.utils import plot_model
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.layers.merge import concatenate
from tensorflow.python.keras.models import model_from_json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#open processed datasets
csv_train=open('processed_data/train_dataset.txt', newline='')
reader_train = csv.reader(csv_train, delimiter=',', quotechar='|')

csv_test=open('processed_data/test_dataset.txt', newline='')
reader_test = csv.reader(csv_test, delimiter=',', quotechar='|')

X_train_dataset=[]
X_test_dataset=[]

Y_train_dataset=[]
Y_test_dataset=[]

In [ ]:
#control variables for input of neural network


target_size=10      #num of position for elements in formula
num_elements=17      # paramertrs of one element (1 coefficeent of element in formula, 16 parametrs of element from chemestry table)


#you can change target_size, but num_elements must have value 17 for these processed data. 
# if you want train network with different parametrs, you must change variable "params_order" in file preparecsvBIG.ipynb and create your won dataset from source data 

In [ ]:
#preprocessing datasets
train_num_fromulas=0
test_num_fromulas=0
counter_tr=0    
read_it=True
while(read_it==True):
    try:
        counter_tr=counter_tr+1
        read_row = next(reader_train)
        crit_temp=read_row[-1]
        Y_train_dataset.append(float(crit_temp))
    
        buf=[]
        for i in range(len(read_row)-1):
            c=read_row[i].split(" ")
            for j in range(len(c)):
                c[j]=float(c[j])
            c=np.asarray(c)
            buf.append(c)
        buf=np.asarray(buf)
        X_train_dataset.append(buf)
        train_num_fromulas=train_num_fromulas+1
    except StopIteration:
        read_it=False
csv_train.close() 

counter_ts=0
read_it=True
while(read_it==True):
    try:
        counter_ts=counter_ts+1
        read_row = next(reader_test)
        crit_temp=read_row[-1]
        Y_test_dataset.append(float(crit_temp))
    
        buf=[]
        for i in range(len(read_row)-1):
            c=read_row[i].split(" ")
            for j in range(len(c)):
                c[j]=float(c[j])
            c=np.asarray(c)
            buf.append(c)
        buf=np.asarray(buf)
        X_test_dataset.append(buf)
        test_num_fromulas=test_num_fromulas+1
    except StopIteration:
        read_it=False
csv_test.close() 


X_test_dataset=np.asarray(X_test_dataset)
X_train_dataset=np.asarray(X_train_dataset)

add_arr=[]
for i in range(num_elements):
    add_arr.append(0)
add_arr=np.asarray(add_arr)

X_test_dataset_processed=[]
Y_test_dataset_processed=[]

X_val_dataset_processed=[]
Y_val_dataset_processed=[]

X_train_dataset_processed=[]
Y_train_dataset_processed=[]


for i in range(X_test_dataset.shape[0]):

    c=X_test_dataset[i].shape[0]
    dist=target_size-X_test_dataset[i].shape[0]
    buf1=[]
    
    if dist>0:
        for j in range(dist):
            buf1.append(add_arr)
    for j in range(X_test_dataset[i].shape[0]):
        buf1.append(X_test_dataset[i][j])
    buf1=np.asarray(buf1)
    X_test_dataset_processed.append(buf1)
    Y_test_dataset_processed.append(Y_test_dataset[i])

for i in range(X_train_dataset.shape[0]):

    c=X_train_dataset[i].shape[0]
    dist=target_size-X_train_dataset[i].shape[0]
    buf1=[]
    
    if dist>0:
        for j in range(dist):
            buf1.append(add_arr)
    for j in range(X_train_dataset[i].shape[0]):
        buf1.append(X_train_dataset[i][j])
    buf1=np.asarray(buf1)
    X_train_dataset_processed.append(buf1)
    Y_train_dataset_processed.append(Y_train_dataset[i])


X_test_dataset_processed=np.asarray(X_test_dataset_processed)
X_train_dataset_processed=np.asarray(X_train_dataset_processed)

In [ ]:


input_shape = (target_size,num_elements)
pad='same'

kernel_size=3
pool_kernel=2

model_in = Input(shape=input_shape)

perm=Permute((2, 1), input_shape=(target_size, num_elements))(model_in)
conv1D010 = Conv1D(32, kernel_size, activation='relu',padding=pad)(perm)
conv1D011 = Conv1D(32, kernel_size ,activation='relu',padding=pad)(conv1D010)
pool011 = MaxPooling1D(pool_size=pool_kernel)(conv1D011)
conv1D012 = Conv1D(64, kernel_size, activation='relu',padding=pad)(pool011)
conv1D013 = Conv1D(64, kernel_size ,activation='relu',padding=pad)(conv1D012)
pool012 = MaxPooling1D(pool_size=pool_kernel)(conv1D013)
conv1D014 = Conv1D(128, kernel_size ,activation='relu',padding=pad)(pool012)
conv1D015 = Conv1D(128, kernel_size ,activation='relu',padding=pad)(conv1D014)
flatten01=Flatten()(conv1D015)

out=Dense(1)(flatten01)
model=Model([model_in], out)


In [ ]:
#training parametrs
loss='mae'#can be mae or mse
optimiser=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
metrics='mae'#can be mae or mse

model.compile(optimizer=optimiser, 
              loss=loss, 
              metrics=[metrics])
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#function for training early stopping when error<value
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True


In [ ]:
optimiser=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=optimiser, 
              loss=loss, 
              metrics=[metrics])
model.summary()
callbacks = [
    EarlyStoppingByLossVal(monitor='val_loss', value=4.9, verbose=1),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    ModelCheckpoint("checkpoint/conv_C3_checkpoint.ckpt", monitor='val_loss', save_best_only=True, verbose=0),
]

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
history = model.fit(X_train_dataset_processed, 
                    Y_train_dataset_processed, 
                    validation_data=(X_test_dataset_processed, Y_test_dataset_processed),
                    epochs=300,
                    batch_size=200,
                    callbacks=callbacks
                    )
print(history.history.keys())

mse, mae = model.evaluate(X_test_dataset_processed, Y_test_dataset_processed, verbose=2)
print("mae mistake:", mae)
print("mistake mse:", mse)

In [ ]:
optimiser=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=optimiser, 
              loss=loss, 
              metrics=[metrics])
model.summary()
callbacks = [
    EarlyStoppingByLossVal(monitor='val_loss', value=4.8, verbose=1),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    ModelCheckpoint("checkpoint/conv_C3_checkpoint.ckpt", monitor='val_loss', save_best_only=True, verbose=0),
]

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
history = model.fit(X_train_dataset_processed, 
                    Y_train_dataset_processed, 
                    validation_data=(X_test_dataset_processed, Y_test_dataset_processed),
                    epochs=300,
                    batch_size=200,
                    callbacks=callbacks
                    )
print(history.history.keys())

mse, mae = model.evaluate(X_test_dataset_processed, Y_test_dataset_processed, verbose=2)
print("mae mistake:", mae)
print("mistake mse:", mse)

In [ ]:
optimiser=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(optimizer=optimiser, 
              loss=loss, 
              metrics=[metrics])
model.summary()
callbacks = [
    EarlyStoppingByLossVal(monitor='val_loss', value=4.8, verbose=1),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    ModelCheckpoint("checkpoint/conv_C3_checkpoint.ckpt", monitor='val_loss', save_best_only=True, verbose=0),
]

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
history = model.fit(X_train_dataset_processed, 
                    Y_train_dataset_processed, 
                    validation_data=(X_test_dataset_processed, Y_test_dataset_processed),
                    epochs=300,
                    batch_size=200,
                    callbacks=callbacks
                    )
print(history.history.keys())

mse, mae = model.evaluate(X_test_dataset_processed, Y_test_dataset_processed, verbose=2)
print("mae mistake:", mae)
print("mistake mse:", mse)

In [ ]:

model_json = model.to_json()
json_file = open('pretrained_networks/conv_C3' + ".json", "w")
#write structure
json_file.write(model_json)
json_file.close()
# write wtights
model.save_weights('pretrained_networks/conv_C3'+".h5")
print("saving done")


In [ ]:
#you can test it for different temperatures

#temperature diapason
min_val=90#minimum temperature
max_val=999#maximum temperature

X_test_0=[]
Y_test_0=[]
print("test")
for i in range(len(Y_test_dataset_processed)):
    if Y_test_dataset_processed[i]>min_val and Y_test_dataset_processed[i]<max_val:
        X_test_0.append(X_test_dataset_processed[i])
        Y_test_0.append(Y_test_dataset_processed[i])
X_test_0=np.asarray(X_test_0)
pred = model.predict(X_test_0)

mistake=0
a=[0,0,0,0,0]
for i in range(len(Y_test_0)):
    mistake=mistake+abs(pred[i][0]-Y_test_0[i])
    if abs(pred[i][0]-Y_test_0[i])<1:
        a[0]=a[0]+1
    if abs(pred[i][0]-Y_test_0[i])>1 and abs(pred[i][0]-Y_test_0[i])<5:
        a[1]=a[1]+1
    if abs(pred[i][0]-Y_test_0[i])>5 and abs(pred[i][0]-Y_test_0[i])<10:
        a[2]=a[2]+1
    if abs(pred[i][0]-Y_test_0[i])>10 and abs(pred[i][0]-Y_test_0[i])<100:
        a[3]=a[3]+1
mistake=mistake/len(Y_test_0)
print("MAE for these temperatures")
print(mistake)

print((a[0]+a[1]+a[2])/((a[0]+a[1]+a[2]+a[3])))
print("percent of error smaller than 1 degree")
print((a[0])/((a[0]+a[1]+a[2]+a[3])))
print("percent of error smaller than 5 degree")
print((a[1])/((a[0]+a[1]+a[2]+a[3])))
print("percent of error smaller than 10 degree")
print((a[2])/((a[0]+a[1]+a[2]+a[3])))

print("show all results")
for i in range(len(Y_test_0)):
    print("preficted="+str(pred[i][0])+" real="+str(Y_test_0[i]))
X_test_0=[]
Y_test_0=[]


